# Ayudantía 11: Networking 🌐
### Autores:
 - Tom Bañados Russell (@tqmbanados)
 - Camila González Piñones (@camilagonzalezp)
 - Pablo Kipreos Palau (@Pablok98)

## Nociones generales

## Sockets
En Python, utilizamos la librería `socket` para programar con Networking. De acá obtenemos el objeto `socket`, que es el que utilizamos para la comunicación por internet, dado que permite enviar y recibir datos entre programas.
- **Ip y puerto**: la dirección de **ip** es la dirección de tu computador, única para él dentro de la red. Es como la dirección de una casa, podemos usarla para llegar a ella. El **puerto** es como la "puerta" por donde se establece la conexión. Un dispotivo tiene una dirección de ip, pero muchos puertos.

### ¿Cómo se usan sockets?

#### **Cliente**

In [ ]:
import socket

# sock será nuestro socket instancia de la clase socket. Este usará dirección IPv4 y protocolo TCP.
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Obtenemos la dirección IP del computador y definimos un puerto de forma arbitraria.
# En este ejemplo, el socket servidor estará en el mismo computador, por eso obtenemos la dirección de esta forma.
# si el servidor está en otro computador, tendríamos que poner su IP correspondiente.
HOST = socket.gethostname()
PORT = 5000

# Ahora intentamos conectarnos a nuestro servidor
# Utilizamos un try porque cuando no se logra la conexión, se levanta una excepción.
try:
    # Con connect intentamos establecer la conexión con otro socket que esté escuchando la dirección y puerto dado.
    sock.connect((HOST, PORT))  # OJO: se tiene que entregar una tupla con los parámetros!

    sock.send('Mensaje bien bacan'.encode())
    # Alternativamente:
    # sock.send(b'Mensaje bien bacan')
    # en general es bueno hacer un protocolo personalizado de comunicación

except ConnectionError as error:
    # Cuando entramos acá, es porque no había un socket escuchando en la dirección dada, o se desconectó en el proceso.
    print('La conexión falló :O')
    print(error)
finally:
    # Dejamos un finally para cerrar el socket, haya error o no. Esto siempre es deseable, como cerrar un archivo!
    sock.close()

#### **Servidor**

In [ ]:
import socket
# sock será nuestro socket instanciado. Usará dirección de tipo ipv4 y será TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Obtenemos la dirección de ip del computador y definimos un puerto de forma arbitraria.
HOST = socket.gethostname()
PORT = 5000

# Utilizamos bind para determinar la dirección y puerto que el socket estará escuchando
sock.bind((HOST, PORT))  # OJO: se tiene que entregar una tupla con los parámetros!
# Ahora si, le decimos al socket que empieze a escuchar conexiones entrantes
sock.listen()

# Como queremos que el servidor esté constantemente aceptando clientes, utilizamos un loop infinito.
# La forma exacta de como manejar esto se ve en los siguientes ejemplos.
while True:
    # Aceptamos la dirección entrante, el loop quedará pegado en esta línea hasta que haya una conexión para aceptar
    socket_client, direccion = sock.accept()  # Mira bien que devuelve el metodo accept!
    # Cuando se acepta la conexión, tenemos una referencia al socket del cliente (para enviarle mensajes y recibirlos)
    # además de su dirección ip!
    print("Conexión aceptada desde", direccion)
    # Imprimimos el mensaje que nos envía el cliente, antes de pasar a escuchar nuevas conexiones
    print(socket_client.recv(2**12))  # Para recibir el mensaje, usamos la referencia al socket del cliente

### Métodos notables

- **`send(data)`**: recibe datos que se quieren enviar a través del `socket`, en forma de `bytes`.
- **`sendall(data)`**: funciona prácticamente igual que `send` pero siempre va a intentar enviar todos los bytes. En general, es recomendable utilizar este método en este curso.
- **`recv(tamaño)`**: este método permite recibir los `bytes` que hayan sido enviados por el otro lado de la conexión (es decir, si un `socket` utiliza `send`, el otro `socket` puede recibir los `bytes` utilizando `recv`). El parámetro `tamaño` es un número (`int`) correspondiente a la cantidad de `bytes` que se quiere recibir.
- **`close()`**: cierra la conexión del `socket`. Esto permite al `socket` realizar una nueva conexión.

#### Para un `socket` con rol de cliente
- **`connect( (ip, puerto) )`**: recibe una `tupla` con la **dirección de ip** y el **puerto** al cual se quiere conectar el `socket` (ambos de tipo `string`)


#### Para un `socket` con rol de servidor

- **`bind( (ip, puerto) )`**: recibe una `tupla` con la **dirección de ip** y el **puerto** al cual se quiere que el `socket` (servidor) escuche conexiones.
- **`listen()`**: le indica al servidor que parta recibiendo conexiones.
- **`accept()`**: le indica al servidor que acepte una conexión entrante. El `thread` que está el servidor quedará pausado hasta que algún cliente se intente conectar, aceptando su conexión cuando esto ocurra y siguiendo. Cabe destacar que este método devuelve una referencia al `socket` conectado con el servidor y su dirección de ip.

*PD: las separaciones por "rol" son solo según su uso general, no es que estos métodos sean exclusivos para ese rol.*

#### Para un socket UDP
- **`sendto(data) y recvfrom(tamaño)`**: equivalentes a los métodos `send` y `recv` respectivamente.

#### Una ayudita mas...
**`socket.gethostname()`** permite obtener la dirección de ip del computador donde se llama este método.

## Serialización: Pickle y JSON

- **Pickle**: exclusivo para *Python*, puede serializar cualquier objeto.
- **JSON**: protocolo estándar para muchos lenguajes, solo puede serializar ciertos tipos de datos. Se estructura de una forma equivalente al diccionario de python.

Estas librerías convierten datos en `bytes`, y ¿sabes que podemos hacer con los `bytes`? ¡Enviarlos por un `socket`!

### Métodos notables

- **`dumps(data)`:** recibe el parámetro `data` que corresponde a algún tipo de datos. Devuelve los datos serializados.
 - **`dump(data, ubicación)`:** equivale al método `dumps`, con la diferencia que crea un archivo serializado en la `ubicación` indicada.
- **`loads(data)`:** recibe el parámetro `data` que corresponde a `bytes` encriptados con `JSON/PICKLE` y devuelve el objeto desencriptado.
 - **`load(ubicacion)`:** equivale al método `loads`, pero recibe la ubicación de un archivo serializado con `dump`.

### Como serializar easy peazy:

In [1]:
import pickle
import json

# Creamos un objeto de python (una clase).
class ObjetoPython:
    def __init__(self, nombre):
        self.nombre = nombre

    def saludar(self):
        print(f"Hola soy: {self.nombre}")

# Creamos un diccionario utilizando tipos genéricos de python
objeto_generico = {
    'ayudante_1': 'Camila',
    2: 'Thom',
    "3": True,
    "cuatro": [1, 2, 3, 4]
}
objeto_python = ObjetoPython("Zanahoria")

# Hora de serializar!
serializacion_pickle = pickle.dumps(objeto_generico)
serializacion_json = json.dumps(objeto_generico)

print(pickle.loads(serializacion_pickle))
print(json.loads(serializacion_json))

# No podemos mezclar serializaciones :O
# print(json.load(serializacion_pickle))
print('Objeto: ', objeto_python)
serializacion_objeto = pickle.dumps(objeto_python)
print('Bytes de la serialización: ', serializacion_objeto)
objeto_deserializado = pickle.loads(serializacion_objeto)
print('Objeto: ', objeto_deserializado)
objeto_deserializado.saludar()

# Ahora veamos que pasa si intentamos serializar un objeto de python con JSON
# serializacion_objeto = json.dumps(objeto_python)

{'ayudante_1': 'Camila', 2: 'Thom', '3': True, 'cuatro': [1, 2, 3, 4]}
{'ayudante_1': 'Camila', '2': 'Thom', '3': True, 'cuatro': [1, 2, 3, 4]}
Objeto:  <__main__.ObjetoPython object at 0x0000018A417E8F48>
Bytes de la serialización:  b'\x80\x03c__main__\nObjetoPython\nq\x00)\x81q\x01}q\x02X\x06\x00\x00\x00nombreq\x03X\t\x00\x00\x00Zanahoriaq\x04sb.'
Objeto:  <__main__.ObjetoPython object at 0x0000018A417C1708>
Hola soy: Zanahoria


## Enviando información: protocolos y *chunks*


En la comuniación entre sockets hay una alta probabilidad de que ocurran errores en el traspaso de información. Para evitar estos errores en lo posible, usamos los llamados **protocolos de comunicación**, y el envío de datos por **_chunks_**. Usando estas herramientas, lograremos una comunicación robusta. 

#### Protocolo
El protocolo de comunicación son las "reglas" sobre el envío de información que se acuerdan entre el *server* y *client*. Este protocolo puede incluir muchas cosas que detallaremos pronto. 
Lo esencial es que el protocolo sea claro y efectivo para transmitir con claridad cada mensaje.

#### Header
Es normal que cada archivo o mensaje parta con un *header* que contiene información sobre su formato, tamaño, y otras cosas importantes para decodificar el archivo o mensaje. 

In [ ]:
mensaje = "¡Hay zombies afuera de mi casa!"
tema1 = "Estoy jugando Minecraft"
tema2 = "Estoy mirando por la ventana"

Lo anterior es un ejemplo de conversaciones entre seres humanos. Vemos que el significado del mensaje cambia drásticamente según el tema de conversación, o análogamente, el *header*. En computadores es algo distinta la comunicación, dado que debemos primero establecer en el protocolo muchas cosas como el **tamaño** del *header* y su **formato**. Como ya dijimos, este protocolo debe ser consistente, o tendremos más errores de comunicación.

#### *Chunks*

Para hacer la comunicación aún más robusta, enviaremos los mensajes en pedazos digeribles y concisos. 
*Para consideración:*
* Un audio de whatsapp de 3 segundos tiene en promedio alredeor de 6000 bytes.
* Una foto de perfil en telegram puede tener hasta 60.000 bytes.
* Este archivo jupyter tiene más de 15.000 bytes. 

Para poder enviar la información de manera controlada y precisa, separamos el archivo en *chunks* de un tamaño **consistente**. **Este tamaño es definido en el protocolo de comunicación.** El *header* debería incluir cuántos *chunks* tiene el mensaje, para así  saber cuando detener la lectura y comenzar a interpretar el mensaje. 

Los mensajes del siguiente ejemplo serán cortos, por lo que usaremos chunks bastante pequeños. Definiremos el siguiente protocolo:
* El tamaño de los *chunks* será de 5 bytes. 
* Cada mensaje comienza con un *header* de 4 bytes.
* Este header contiene la cantidad de chunks del mensaje codificada en *big endian*.
* Todos los chunks deben tener el mismo tamaño. Para esto, se rellena con 0s el final del mensaje hasta que el último chunk tenga el tamaño correcto.
* Codificamos nuestros strings en utf-8.

In [1]:
CHUNK_SIZE = 5

mensaje = "Hola soy un mensaje! Porfavor recíbeme correctamente uwu. En caso de que ocurra un error explotaré!"
bytes_mensaje = mensaje.encode(encoding='UTF-8')
# Cada chunk debe tener el mismo tamaño!
while len(bytes_mensaje) % CHUNK_SIZE != 0:
    bytes_mensaje += b'\x00'
n_chunks = len(bytes_mensaje) // CHUNK_SIZE

n_chunks_bytes = n_chunks.to_bytes(4, "big")
mensaje_final = n_chunks_bytes + bytes_mensaje
# Nuestro mensaje está listo para enviarse!

In [2]:
# Ahora definiremos las funciones que reciben los mensajes por chunks
def recibir_mensaje(nuevo_mensaje):
    chunks_recibidos = []
    n_chunks = int.from_bytes(nuevo_mensaje[:4], "big")
    actual = 4
    while len(chunks_recibidos) < n_chunks:
        chunk = nuevo_mensaje[actual:actual + CHUNK_SIZE]
        print(chunk)
        chunks_recibidos.append(chunk)
        actual += CHUNK_SIZE
    completar_menaje(chunks_recibidos)

def completar_menaje(chunks_recibidos):
    mensaje_final = b"".join(chunks_recibidos).strip(b"\x00")
    mensaje_decodificado = mensaje_final.decode(encoding="UTF-8").strip()
    print("Recibí el siguiente mensaje:\n", mensaje_decodificado)

In [3]:
# Veamos que pasa!
recibir_mensaje(mensaje_final)

b'Hola '
b'soy u'
b'n men'
b'saje!'
b' Porf'
b'avor '
b'rec\xc3\xad'
b'beme '
b'corre'
b'ctame'
b'nte u'
b'wu. E'
b'n cas'
b'o de '
b'que o'
b'curra'
b' un e'
b'rror '
b'explo'
b'tar\xc3\xa9'
b'!\x00\x00\x00\x00'
Recibí el siguiente mensaje:
 Hola soy un mensaje! Porfavor recíbeme correctamente uwu. En caso de que ocurra un error explotaré!    


##### Información que podría contener un protocolo
* Tamaño de los chunks.
* Tamaño del header.
* Formato del header.
* Codificación según little endian o big endian.
* Codificación de strings (ascii, utf-8, etc.).
* Formato de cada chunk (si es que parte con el número de chunk, y cuantos bytes son destinados a esto).

##### Información que podría contener un header
* Contenido del mensaje (por ejemplo el formato: wav, bmp, jpg...).
* Cantidad de chunks.
* Cualquier otra información importante para la decodificación.

### Chunk + JSON = profit
Utilizando serialización, las librerías pickle o JSON y un protocolo con *chunks*, podemos tener una comunicación robusta.
A continuación, aplicaremos lo repasado para hacer un traspaso de información algo más "real". 
Usemos el siguiente protocolo:
* El tamaño de los *chunks* será de 10 bytes. 
* Cada mensaje comienza con un *header* de 8 bytes.
* Este header contiene 4 bytes que indican la cantidad de chunks del mensaje codificada en *little endian*.
* Los siguientes 4 bytes indican el tipo de mensaje recibido.
* Todos los *header* deben tener el mismo tamaño. Para esto, se rellena con 0s el final del header hasta que su tamaño sea 8.
* Todos los chunks deben tener el mismo tamaño. Para esto, se rellena con 0s el final del mensaje hasta que el último chunk tenga el tamaño correcto.
* Codificamos nuestros strings en utf-8.

In [4]:
import json

HEADER_SIZE = 8
CHUNK_SIZE = 10
# Supongamos la siguiente "base de datos"
json_data = {"MP3": {"Baby Shark": {"NOMBRE": "Baby Shark",
                                    "MUSICA": "du duru duru durú"},
                     "Safaera": {"NOMBRE": "Safaera",
                                 "MUSICA": "Aqui llegó tu tiburón"}
                     },
             "MEME": {"original_meme": {
                 "IMAGEN": "HACKERMAN",
                 "TOP TEXT": "Cuando tienes que inventar un meme para la ayudantía",
                 "BOTTOM TEXT": "bottom text"}}
             }

In [5]:
# Definimos dos funciones para los dos tipos de archivo que tiene nuestra base de datos
def leer_meme(meme):
    print(f"""Recibimos un meme!
            {meme["TOP TEXT"]}
            {meme["IMAGEN"]}
            {meme["BOTTOM TEXT"]}
          """)

def leer_cancion(cancion):
    print("recibimos una canción!")
    print(f"Ahora escucharemos {cancion['NOMBRE']}:")
    print(cancion["MUSICA"])

In [6]:
# Crearemos una clase que recibe mensajes y lo lee según el protocolo definido
class Reader:
    def __init__(self):
        print("estoy esperando que me envíen datos!")

    def recibir_mensaje(self, data_):
        header = data_[:HEADER_SIZE].strip(b'\x00')
        n_chunks = int.from_bytes(header[:4], "little")
        formato = header[4:].decode('utf-8')
        chunks_recibidos = []
        pos_actual = HEADER_SIZE
        while len(chunks_recibidos) < n_chunks:
            chunk = data_[pos_actual:pos_actual + CHUNK_SIZE]
            chunks_recibidos.append(chunk)
            pos_actual += CHUNK_SIZE
        mensaje = b"".join(chunks_recibidos).strip(b"\x00")
        self.leer_mensaje(mensaje, formato)

    def leer_mensaje(self, mensaje_json, formato):
        mensaje = json.loads(mensaje_json, encoding='utf-8')
        if formato == 'MEME':
            leer_meme(mensaje)
        elif formato == 'MP3':
            leer_cancion(mensaje)
        else:
            print("formato:", formato)
            raise ValueError("No sé que hacer con esta información unu")

In [7]:
# Ahora creamos una funcion que recibe la información, y la codifica correctamente
def codificar(formato, data_):
    info_codificada = json.dumps(data_).encode(encoding='utf-8')
    while len(info_codificada) % CHUNK_SIZE != 0:
        info_codificada += b'\x00'

    n_chunks = len(info_codificada) // CHUNK_SIZE
    header = n_chunks.to_bytes(4, "little") + formato.encode('utf-8')
    while len(header) % HEADER_SIZE != 0:
        header += b'\x00'
    return header + info_codificada

In [8]:
# Probemos nuestro código!
server_ficticio = Reader()
mensaje1 = codificar("MP3", json_data["MP3"]["Baby Shark"])
mensaje2 = codificar("MEME", json_data["MEME"]["original_meme"])
server_ficticio.recibir_mensaje(mensaje1)
server_ficticio.recibir_mensaje(mensaje2)

estoy esperando que me envíen datos!
recibí una canción!
Ahora escucharemos Baby Shark:
du duru duru durú
Recibí un meme!
            Cuando tienes que inventar un meme para la ayudantía
            HACKERMAN
            bottom text
          


Lo anterior han sido ejemplos "aislados". No son verdaderamente útiles hasta ponerse en la práctica de comunicación vía socket. A continuación revisaremos un ejemplo más completo, que resume todo lo visto hasta ahora!

## Threading... otra vez 👀
Por si aún no estabas convencido(a) de que **Threading** era importante, bueno aquí tienes nuestro último argumento: **¡threading en networking!**

Como vimos, nos gustaría que el servidor pueda escuchar nuevas conexiones y procesar los mensajes de todos los clientes... todo al mismo tiempo. Esto
está pintado para concurrencia 😱. Para solucionar esto, hacemos que el servidor cree un thread nuevo para cada cliente conectado, el cual se preocupará de
ir escuchando y procesando sus mensajes respectivos. Por otro lado, dejamos el loop que acepta nuevas conexiones dentro del thread principal.

### Aceptando conexiones recurrentemente y entendiendo como usar `accept`

In [ ]:
import json
import socket
import threading


class Servidor:

    def __init__(self, host, port):
        # host en el que está corriendo el servidor
        self.host = host
        # puerto donde se escucharán las conexiones
        self.port = port
        # Crear socket con dirección IPv4 y protocolo TCP
        print("Iniciando servidor...")
        self.socket_servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # Ligar host y port
        self.socket_servidor.bind((self.host, self.port))
        # Empezar a escuchar conexiones
        self.socket_servidor.listen()
        # Crear y empezar thread encargado de aceptar conexiones
        thread = threading.Thread(target=self.aceptar_clientes)
        thread.start()

    def aceptar_clientes(self):
        print("Escuchando clientes...")
        while True:
            # Comenzar a aceptar clientes
            # (socket, (dirección IP, puerto)) = accept()
            socket_cliente, address = self.socket_servidor.accept()
            print("Conexión aceptada: ", address)
            # Crear y empezar un thread para escuchar a cada cliente
            thread_cliente = threading.Thread(target=self.escuchar_cliente, args=(socket_cliente,), daemon=True)
            thread_cliente.start()
    
    def escuchar_cliente(self, socket_cliente):
        try:
            while True:
                # Recibir y manejar mensajes del cliente
                mensaje = self.recibir(socket_cliente)
                print("Mensaje recibido: ", mensaje)
                if mensaje != "":
                    respuesta = self.manejar_mensaje_recibido(mensaje, socket_cliente)
                    self.enviar(respuesta, socket_cliente)
        except ConnectionResetError:
            print("Error de conexión con el cliente")

    def recibir(self, socket_cliente):
        # Recibir largo del mensaje
        largo_bytes_mensaje = socket_cliente.recv(4)
        # Decodificar largo del mensaje
        largo_mensaje = int.from_bytes(largo_bytes_mensaje, byteorder="big")
        # Recibir mensaje
        bytes_mensaje = bytearray()
        while len(bytes_mensaje) < largo_mensaje:
            bytes_a_recibir = min(4096, largo_mensaje - len(bytes_mensaje))
            bytes_mensaje += socket_cliente.recv(bytes_a_recibir)
        # Decodificar mensaje
        mensaje = self.decodificar_mensaje(bytes_mensaje)
        return mensaje

    def enviar(self, mensaje, socket_jugador):
        # Codificar mensaje
        bytes_mensaje = self.codificar_mensaje(mensaje)
        # Codificar largo del mensaje
        largo_bytes_mensaje = len(bytes_mensaje).to_bytes(4, byteorder="big")
        # Enviar largo del mensaje + mensaje
        socket_jugador.sendall(largo_bytes_mensaje + bytes_mensaje)

    def codificar_mensaje(self, mensaje):
        try:
            # Serializar el mensaje
            mensaje_json = json.dumps(mensaje)
            # Codificar el mensaje serializado
            bytes_mensaje = mensaje_json.encode("utf-8")
            return bytes_mensaje
        except json.JSONDecodeError:
            print("No se pudo codificar el mensaje")
            return b""

    def decodificar_mensaje(self, bytes_mensaje):
        try:
            # Deserializar el mensaje
            mensaje = json.loads(bytes_mensaje)
            return mensaje
        except json.JSONDecodeError:
            print("No se pudo decodificar el mensaje")
            return dict()

    def manejar_mensaje_recibido(self, mensaje, socket_cliente):
        # De acuerdo al mensaje que se reciba de parte del cliente se debería generar una respuesta acorde
        respuesta = f"[Respuesta de servidor asociada al mensaje: {mensaje}]"
        print("Enviando respuesta al cliente: ", respuesta)
        return respuesta


if __name__ == "__main__":
    host = "localhost"
    port = 5000
    servidor = Servidor(host, port)
    

In [ ]:
import json
import socket
import threading


class Cliente:

    def __init__(self, host, port):
        # Dirección IP del host destinatario
        self.host = host
        # Puerto al que nos queremos conectar
        self.port = port
        # Crear socket con dirección IPv4 y protocolo TCP
        self.socket_cliente = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            # Conectarse al servidor
            self.socket_cliente.connect((self.host, self.port))
            # Crear y empezar thread encargado de escuchar al servidor
            thread = threading.Thread(target=self.escuchar_servidor, daemon=True)
            thread.start()
            print("Conexión exitosa")
            # Ingresar mensaje a enviar al servidor
            self.recibir_input()
        except ConnectionRefusedError:
            print("Error en la conexión")
            self.socket_cliente.close()

    def escuchar_servidor(self):
        try:
            while True:
                # Recibir y manejar mensajes del servidor
                mensaje = self.recibir()
                print("Mensaje recibido del servidor: ", mensaje)
                if mensaje != "":
                    self.manejar_mensaje_recibido(mensaje)
        except ConnectionResetError:
            print("Error de conexión con el servidor")
        finally:
            # Cerrar socket
            self.socket_cliente.close()

    def recibir(self):
        # Recibir largo del mensaje
        largo_bytes_mensaje = self.socket_cliente.recv(4)
        # Decodificar largo del mensaje
        largo_mensaje = int.from_bytes(largo_bytes_mensaje, byteorder="big")
        # Recibir mensaje
        bytes_mensaje = bytearray()
        while len(bytes_mensaje) < largo_mensaje:
            bytes_a_recibir = min(4096, largo_mensaje - len(bytes_mensaje))
            bytes_mensaje += self.socket_cliente.recv(bytes_a_recibir)
        # Decodificar mensaje
        mensaje = self.decodificar_mensaje(bytes_mensaje)
        return mensaje

    def enviar(self, mensaje):
        print("Enviando al servidor el mensaje: ", mensaje)
        # Codificar mensaje
        bytes_mensaje = self.codificar_mensaje(mensaje)
        # Codificar largo del mensaje
        largo_bytes_mensaje = len(bytes_mensaje).to_bytes(4, byteorder="big")
        # Enviar largo del mensaje + mensaje
        self.socket_jugador.sendall(largo_bytes_mensaje + bytes_mensaje)

    def codificar_mensaje(self, mensaje):
        try:
            # Serializar el mensaje
            mensaje_json = json.dumps(mensaje)
            # Codificar el mensaje serializado
            bytes_mensaje = mensaje_json.encode("utf-8")
            return bytes_mensaje
        except json.JSONDecodeError:
            print("No se pudo codificar el mensaje")
            return b""

    def decodificar_mensaje(self, bytes_mensaje):
        try:
            # Deserializar el mensaje
            mensaje = json.loads(bytes_mensaje)
            return mensaje
        except json.JSONDecodeError:
            print("No se pudo decodificar el mensaje")
            return dict()

    def manejar_mensaje_recibido(self, mensaje):
        # De acuerdo al mensaje que se reciba de parte del servidor se debería generar una acción acorde.
        accion = f"Realizando acción asociada al mensaje: {mensaje}"
        print(accion) 

    def recibir_input(self):
        while True:
            mensaje = input()
            self.enviar(mensaje)


if __name__ == "__main__":
    host = "localhost"
    port = 5000
    cliente = Cliente(host, port)